In [7]:
# Install needed libraries
%pip install -U python-jobspy
%pip install tqdm
%pip install xlsxwriter

You should consider upgrading via the '/home/wagner/.pyenv/versions/3.10.0/envs/market_scrapper_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/wagner/.pyenv/versions/3.10.0/envs/market_scrapper_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/wagner/.pyenv/versions/3.10.0/envs/market_scrapper_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
from jobspy import scrape_jobs
from pathlib import Path 
from datetime import datetime

In [15]:
def setup_output_directory(base_dir="../data/raw"):
    """
    Crea un directorio de salida con timestamp (usando pathlib) 
    y devuelve un objeto Path.
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 2. Usar pathlib para crear la ruta
    output_dir = Path(base_dir) / f"jobs_{timestamp}"
    
    # 3. Usar .mkdir() del objeto Path
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 4. Devolver el objeto Path (no un string)
    return output_dir

In [16]:
# --- 1. Definir Directorio de Salida ---
output_dir = setup_output_directory()
print(f"Directorio de salida: {output_dir}")

Directorio de salida: ../data/raw/jobs_20251115_160526


In [ ]:
# --- 2. Definir Parámetros de Búsqueda Base ---
roles_buscados = ["Data Scientist", "Machine Learning Engineer", "Data Analyst"]
sectores_clave = ["Fintech", "EdTech", "Future of Work"]
terminos_busqueda = list(product(roles_buscados, sectores_clave))
search_term_base = f'({rol_buscado}) AND ({sectores_clave})'
location = "Remote"
country_code = "USA" # Código de país para Indeed

In [18]:
# --- 3. Lista para guardar resultados ---
# Guardaremos los DataFrames de cada sitio aquí
all_jobs_dfs = []

print("Parámetros listos. Iniciaremos scrapers secuenciales y especializados.")

Parámetros listos. Iniciaremos scrapers secuenciales y especializados.


In [19]:
# --- 1. Scraper: Indeed (El "Caballo de batalla") ---
# Es el más estable y sin límites de solicitudes
print("\n--- Iniciando Scraper: Indeed ---")
print(f"Buscando: {search_term_base} | Resultados: 200")
try:
    indeed_jobs = scrape_jobs(
        site_name=["indeed"],
        search_term=search_term_base + " -marketing", # Podemos afinar el término
        location=location,
        country_indeed=country_code,
        results_wanted=200, # Le pedimos más porque es estable
        hours_old=720
    )
    if indeed_jobs is not None and not indeed_jobs.empty:
        print(f"✅ Indeed encontró {len(indeed_jobs)} trabajos.")
        all_jobs_dfs.append(indeed_jobs)
except Exception as e:
    print(f"❌ Error en Indeed: {e}")


--- Iniciando Scraper: Indeed ---
Buscando: ("Data Scientist" OR "Machine Learning Engineer" OR "Data Analyst") AND ((Fintech OR EdTech OR "Future of Work")) | Resultados: 200
✅ Indeed encontró 29 trabajos.


In [20]:
# --- 2. Scraper: LinkedIn (El "Delicado") ---
# Alto riesgo de 429. Lo llamamos con cuidado.
print("\n--- Iniciando Scraper: LinkedIn ---")
print(f"Buscando: {search_term_base} | Resultados: 50 (Límite bajo sin proxy)")
try:
    linkedin_jobs = scrape_jobs(
        site_name=["linkedin"],
        search_term=search_term_base,
        location=location,
        results_wanted=50, # MUY BAJO para evitar 429 sin proxies
        hours_old=720,
        linkedin_fetch_description=True # Clave para enriquecimiento
    )
    if linkedin_jobs is not None and not linkedin_jobs.empty:
        print(f"✅ LinkedIn encontró {len(linkedin_jobs)} trabajos.")
        all_jobs_dfs.append(linkedin_jobs)
except Exception as e:
    print(f"❌ Error en LinkedIn: {e}. (Probablemente Error 429)")



--- Iniciando Scraper: LinkedIn ---
Buscando: ("Data Scientist" OR "Machine Learning Engineer" OR "Data Analyst") AND ((Fintech OR EdTech OR "Future of Work")) | Resultados: 50 (Límite bajo sin proxy)
✅ LinkedIn encontró 50 trabajos.


In [21]:

# --- 3. Scraper: ZipRecruiter (El "Estándar") ---
print("\n--- Iniciando Scraper: ZipRecruiter ---")
print(f"Buscando: {search_term_base} | Resultados: 100")
try:
    zip_jobs = scrape_jobs(
        site_name=["zip_recruiter"],
        search_term=search_term_base,
        location=location,
        results_wanted=100, 
        hours_old=720
    )
    if zip_jobs is not None and not zip_jobs.empty:
        print(f"✅ ZipRecruiter encontró {len(zip_jobs)} trabajos.")
        all_jobs_dfs.append(zip_jobs)
except Exception as e:
    print(f"❌ Error en ZipRecruiter: {e}")

print("\n--- Scraping secuencial completado ---")


--- Iniciando Scraper: ZipRecruiter ---
Buscando: ("Data Scientist" OR "Machine Learning Engineer" OR "Data Analyst") AND ((Fintech OR EdTech OR "Future of Work")) | Resultados: 100

--- Scraping secuencial completado ---


In [22]:
if all_jobs_dfs:
    combined_jobs = pd.concat(all_jobs_dfs, ignore_index=True).drop_duplicates(
        subset=['job_url', 'title', 'company']
    )
    
    # Esta línea ahora funciona, porque 'output_dir' es un objeto Path
    combined_filename = f"{output_dir }/all_jobs_combined.json"
    
    combined_jobs.to_json(
        combined_filename, 
        orient='records', 
        indent=4, 
        force_ascii=False
    )
    print(f"\n🎉 ¡Éxito! Se guardaron {len(combined_jobs)} trabajos únicos en:")
    print(combined_filename)
else:
    print("\nNo se pudo descargar ningún trabajo de ninguna fuente.")


🎉 ¡Éxito! Se guardaron 79 trabajos únicos en:
../data/raw/jobs_20251115_160526/all_jobs_combined.json


/tmp/ipykernel_464921/2053293427.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_jobs = pd.concat(all_jobs_dfs, ignore_index=True).drop_duplicates(
